In [16]:
#default_exp environments

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Environments

> Classes to create simulated environment functions.

In [18]:
#%nbdev_hide
#import sys
#sys.path.append("..")

In [19]:
#export
import gym
import math
import numpy as np
import pct.putils as vid
from pct.functions import BaseFunction
from pct.putils import FunctionsList

In [20]:
#export
class EnvironmentFactory:
    factories = {}
    def addFactory(id, environmentFactory):
        EnvironmentFactory.factories.put[id] = environmentFactory
    addFactory = staticmethod(addFactory)
    
    # A Template Method:
    def createEnvironment(id):
        if not EnvironmentFactory.factories.__contains__(id):
            EnvironmentFactory.factories[id] = \
              eval(id + '.Factory()')
        return EnvironmentFactory.factories[id].create()
    
    createEnvironment = staticmethod(createEnvironment)       
    
    
    def createEnvironmentWithNamespace(sid, namespace=None):
        id = sid + f'.FactoryWithNamespace()'                
        if not EnvironmentFactory.factories.__contains__(id):
            EnvironmentFactory.factories[id] = eval(id)
        return EnvironmentFactory.factories[id].create(namespace=namespace)
    createEnvironmentWithNamespace = staticmethod(createEnvironmentWithNamespace)

In [21]:
#export
class OpenAIGym(BaseFunction):
    "A function that creates an runs an environment from OpenAI Gym. Parameter: The environment name. Flag to display environment. Links: Link to the action function."
    def __init__(self, env_name=None, render=False, video_wrap=False, value=0, name="gym", 
                 seed=None, links=None, new_name=True, early_termination=False, namespace=None, **cargs):
        super().__init__(name=name, value=value, links=links, new_name=new_name, namespace=namespace)
        self.early_termination=early_termination
        self.video_wrap = video_wrap
        self.env_name=env_name
        self.max_episode_steps=4000
        self.create_env(seed)
        self.render = render
        self.reward = 0
        self.done = False
        self.info = {}
    
    def __call__(self, verbose=False):
        super().check_links(1)
        self.early_terminate()
        self.input = self.links[0].get_value()
        self.process_input()
        self.obs = self.env.step(self.input)
            
        self.value = self.obs[0]
        self.reward = self.obs[1]
        self.done = self.obs[2]
        self.info = self.obs[3]

        self.process_values()
        out = super().__call__(verbose)
        
        if self.render:
            self.env.render()
            
        return out 

    
    def set_video_wrap(self, video_wrap):
        self.video_wrap = video_wrap

        
    def early_terminate(self):
        if self.early_termination:
            if self.done:
                raise Exception(f'1000: OpenAIGym Env: {self.env_name} has terminated.')
    
    def process_input(self):
        pass
    
    def process_values(self):
        pass

    def set_render(self, render):
        self.render=render
        
    def reset(self, full=True):
        if full:
            super().reset()        
        else:
            self.value=0
        self.really_done = False
        return self.env.reset()


    def summary(self):
        super().summary("")

    def get_config(self):
        "Return the JSON  configuration of the function."
        config = {"type": type(self).__name__,
                    "name": self.name}
        
        if isinstance(self.value, np.ndarray):
            config["value"] = self.value.tolist()
        else:
            config["value"] = self.value
        
        ctr=0
        links={}
        for link in self.links:
            func = FunctionsList.getInstance().get_function(self.namespace, link)
            try:
                links[ctr]=func.get_name()
            except AttributeError:
                #raise Exception(f' there is no function called {link}, ensure it exists first.')            
                print(f'WARN: there is no function called {link}, ensure it exists first.')            
                links[ctr]=func
                
            ctr+=1
        
        config['links']=links

        config["env_name"] = self.env_name
        #config["values"] = self.value
        config["reward"] = self.reward
        config["done"] = self.done
        config["info"] = self.info
        
        return config
    
    def get_reward(self):
        return self.reward

    def output_string(self):
        
        if isinstance(self.value, int):
            rtn = f'{round(self.value, self.decimal_places):.{self.decimal_places}f}'
        else:
            list = [f'{round(val, self.decimal_places):.{self.decimal_places}f} ' for val in self.value]
            list.append(str(self.reward))
            list.append(" ")
            list.append(str(self.done))
            list.append(" ")
            list.append(str(self.info))
            
            rtn = ''.join(list)

        return rtn

    
    def create_env(self, seed):
        genv = gym.make(self.env_name)
        genv._max_episode_steps = self.max_episode_steps
        if self.video_wrap:
            self.env =  vid.wrap_env(genv)
        else:
            self.env = genv
        self.env.reset(seed=seed)
        #self.env.seed(seed)
        self.env.reset()
            
    def set_seed(self, seed):
        self.env.reset(seed=seed)
        #self.env.seed(seed)

                
    def get_parameters_list(self):
        return [self.name]    
        
    def close(self):
        self.env.close()
        
    class Factory:
        def create(self): return OpenAIGym()
        
    class FactoryWithNamespace:
        def create(self, namespace=None): return OpenAIGym(namespace=namespace)        

In [22]:
#export
class CartPoleV1(OpenAIGym):
    "A function that creates an runs the CartPole-v1 environment from OpenAI Gym. Parameter: The environment name. Flag to display environment. Links: Link to the action function."
    # from obs[0], indices
    # 1 cart_velocity
    # 0 cart_position
    # 3 pole_velocity
    # 2 pole_angle
    def __init__(self, render=False, video_wrap=False, value=0, name="CartPoleV1", 
                 seed=None, links=None, new_name=True, namespace=None, **cargs):
        super().__init__(env_name='CartPole-v1', render=render, video_wrap=video_wrap, value=value, name=name, seed=seed, 
                         links=links, new_name=new_name, namespace=namespace, **cargs)
 
    def __call__(self, verbose=False):
        super().__call__(verbose)
        
        return self.value
    
    def process_values(self):
        self.value = np.append(self.value, self.obs[0][0]+math.sin(self.obs[0][2]))
    
    def process_input(self):
        if self.input<0:
            self.input=0
        elif self.input>0:
            self.input=1
        else:
            self.input=0

    class Factory:
        def create(self): return CartPoleV1()
    class FactoryWithNamespace:
        def create(self, namespace=None): return CartPoleV1(namespace=namespace)        

In [23]:
#export
class CartPoleDV1(OpenAIGym):
    "A function that creates an runs the CartPole-v1 environment from OpenAI Gym. Parameter: The environment name. Flag to display environment. Links: Link to the action function."
    # from obs[0], indices
    # 1 cart_velocity
    # 0 cart_position
    # 3 pole_velocity
    # 2 pole_angle
    def __init__(self, render=False, video_wrap=False, value=0, name="CartPoleD-v1", 
                 seed=None, links=None, new_name=True, namespace=None, **cargs):
        super().__init__('CartPoleD-v1', render=render, video_wrap=video_wrap, value=value, name=name, seed=seed, 
                         links=links, new_name=new_name, **cargs)
 
    def __call__(self, verbose=False):
        super().__call__(verbose)

        return self.value
    
    def process_values(self):
        self.value = np.append(self.value, self.obs[0][0]+math.sin(self.obs[0][2]))
        self.value = np.append(self.value, self.env.gravity)
    
    def process_input(self):
        if self.input<0:
            self.input=0
        elif self.input>0:
            self.input=1
        else:
            self.input=0

    class Factory:
        def create(self): return CartPoleDV1()
    class FactoryWithNamespace:
        def create(self, namespace=None): return CartPoleDV1(namespace=namespace)        

In [24]:
#export
class Pendulum(OpenAIGym):
    "A function that creates an runs the Pendulum-v1 environment from OpenAI Gym. Parameter: The environment name. Flag to display environment. Links: Link to the action function."
    # from obs[0], indices
    # 0 cos(theta) - +1 is up, -1 is down, 0 is left and right
    # 1 sin(theta) - +1 is left, -1 is right, 0 is up and down
    # 2 theta dot - +dot is anti-clockwise, -dot is clockwise
    # 3 theta +pi/-pi (added here) 0 is pointing upwards, +pi is anti-clockwise, -pi is clockwise
    # 4 theta +x+pi/x-pi (added here) 0 is pointing downwards, + is anti-clockwise, - is clockwise
    # reward - -(theta^2 + 0.1*theta_dt^2 + 0.001*action^2)

    def __init__(self, render=False, video_wrap=False, value=0, name="Pendulum", 
                 seed=None, links=None, new_name=True, namespace=None,**cargs):        
        super().__init__('Pendulum-v1', render=render, video_wrap=video_wrap, value=value, name=name, seed=seed, 
                         links=links, new_name=new_name, **cargs)
        
    def __call__(self, verbose=False):
        super().check_links(1)
        self.input = self.links[0].get_value()
        self.obs = self.env.step([self.input])
            
        self.value = self.obs[0]
        self.reward = -self.obs[1]
        self.done = self.obs[2]
        self.info = self.obs[3]
        
        pi = math.copysign(math.acos(self.obs[0][0]), self.obs[0][1])
        self.value = np.append(self.value, pi)
        #x = math.copysign(pi-abs(pi), pi)
        x = 10 + pi
        self.value = np.append(self.value, x)
        
        
        if self.render:
            self.env.render()
                
        if verbose :
            print(self.output_string())

        return self.value

    class Factory:
        def create(self): return Pendulum()
    class FactoryWithNamespace:
        def create(self, namespace=None): return Pendulum(namespace=namespace)                

In [25]:
#export
class Pendulum_1(OpenAIGym):
    "A function that creates an runs the Pendulum-v1 environment from OpenAI Gym. Parameter: The environment name. Flag to display environment. Links: Link to the action function."
    # from obs[0], indices
    # 0 cos(theta) - +1 is up, -1 is down, 0 is left and right
    # 1 sin(theta) - +1 is left, -1 is right, 0 is up and down
    # 2 theta dot - +dot is anti-clockwise, -dot is clockwise
    # 3 theta dot - normalised to +/- 1    
    # 4 theta +1/-1 (added here) 1 is pointing upwards, + is anti-clockwise, - is clockwise
    # reward - -(theta^2 + 0.1*theta_dt^2 + 0.001*action^2)

    def __init__(self, render=False, video_wrap=False, value=0, name="Pendulum-1", 
                 seed=None, links=None, new_name=True, namespace=None,**cargs):        
        super().__init__('Pendulum-v1', render=render, video_wrap=video_wrap, value=value, name=name, seed=seed, 
                         links=links, new_name=new_name, **cargs)
        
    def __call__(self, verbose=False):
        super().check_links(1)
        self.input = self.links[0].get_value()
        self.obs = self.env.step([self.input])
            
        self.value = self.obs[0]
        self.reward = -self.obs[1]
        self.done = self.obs[2]
        self.info = self.obs[3]
        
        vel = self.obs[0][2]/8.0
        self.value = np.append(self.value, vel)
        x = math.copysign(math.acos(self.obs[0][0]), self.obs[0][1])/math.pi
        #theta = 100 - (10 * math.copysign(1-abs(x), x))
        theta = 100 - (10 * x)
        self.value = np.append(self.value, theta)
        
        if self.render:
            self.env.render()
                
        if verbose :
            print(self.output_string())

        return self.value

    class Factory:
        def create(self): return Pendulum_1()
    class FactoryWithNamespace:
        def create(self, namespace=None): return Pendulum_1(namespace=namespace)                

In [26]:
#export
class MountainCarV0(OpenAIGym):
    "A function that creates and runs the MountainCar-v0 environment from OpenAI Gym. Parameter: The environment name. Flag to display environment. Links: Link to the action function."
    # from obs[0], indices
    # 0 Car position - -1.2 to +0.6, reference 0.45 
    # 1 Car Velocity - -0.07 t0 +0.07
    # 2 Car position - 0 to +1.8, reference 1.65 
    
    def __init__(self, render=False, video_wrap=False, value=0, name="MountainCarV0", 
                 seed=None, links=None, new_name=True, namespace=None, **cargs):        
        super().__init__('MountainCar-v0', render=render, video_wrap=video_wrap, value=value, name=name, seed=seed, 
                         links=links, new_name=new_name, **cargs)
        
    def __call__(self, verbose=False):        
        super().__call__(verbose)
                
        return self.value

    def process_values(self):        
        self.reward = -self.obs[1]
        pos = self.value[0] + 1.2
        self.value = np.append(self.value, pos)

    def process_input(self):
        if self.input<0:
            self.input=0
        elif self.input>0:
            self.input=2
        else:
            self.input=1

    class Factory:
        def create(self): return MountainCarV0()
    class FactoryWithNamespace:
        def create(self, namespace=None): return MountainCarV0(namespace=namespace)                

In [27]:
#export
class MountainCarContinuousV0(OpenAIGym):
    "A function that creates and runs the MountainCarContinuous-v0 environment from OpenAI Gym. Parameter: The environment name. Flag to display environment. Links: Link to the action function."
    # 0 Car position - -1.2 to +0.6, reference 0.45 
    # 1 Car Velocity - -0.07 t0 +0.07
    # 2 Car position - 0 to +1.8, reference 1.65 
    
    def __init__(self, render=False, video_wrap=False, value=0, name="MountainCarContinuousV0", 
                 seed=None, links=None, new_name=True, early_termination=True, namespace=None, **cargs):        
        super().__init__('MountainCarContinuous-v0', render=render, video_wrap=video_wrap, value=value, name=name, seed=seed, 
                         links=links, new_name=new_name, early_termination=early_termination, **cargs)
        
        self.min_action = -1.0
        self.max_action = 1.0     
        self.really_done = False
        
    def __call__(self, verbose=False):        
        super().__call__(verbose)
                
        return self.value

    def early_terminate(self):
        if self.early_termination:
            if self.really_done:
                raise Exception(f'1000: OpenAIGym Env: {self.env_name} has terminated.')
            if self.done:
                self.reward = 0
                self.really_done = True
                
    def process_input(self):
        force = min(max(self.input, self.min_action), self.max_action)
        self.input=[force]
        
    def process_values(self):
        reward = self.obs[1]
        if reward > 90:
            reward = 0
        self.reward = - reward
        pos = self.value[0] + 1.2
        self.value = np.append(self.value, pos)

    class Factory:
        def create(self): return MountainCarContinuousV0()
    class FactoryWithNamespace:
        def create(self, namespace=None): return MountainCarContinuousV0(namespace=namespace)               

In [28]:
#export
class VelocityModel(BaseFunction):
    "A simple model of a moving object of a particular mass. Parameters: The environment name, mass. Links: Link to the action function."
    # from obs[0], indices
    
    def __init__(self, mass=50, value=0, name="VelocityModel", links=None, 
                 num_links=1, new_name=True, indexes=0, namespace=None, **cargs):        
        super().__init__(name=name, value=value, links=links, new_name=new_name, namespace=namespace)
        self.mass = mass
        self.indexes=indexes
        self.num_links=num_links
        self.init_value()

    
    def __call__(self, verbose=False):
        super().check_links(self.num_links)
        force = self.links[0].get_value()

        if self.indexes>0:
            output = self.value[0] + force / self.mass
            self.value = [output for _ in range(self.indexes)]
        else:        
            self.value = self.value + force / self.mass
        return super().__call__(verbose)

    def init_value(self):
        if self.indexes>0:
            self.value = [1 for _ in range(self.indexes)]
        
        
    def reset(self, full=True):
        if full:
            super().reset()        
        else:
            self.init_value()

        return True

    def set_render(self, render):
        pass
    
    def summary(self):
        super().summary("")
        
    def get_parameters_list(self):
        return ['vm']    

    def get_config(self):
        config = super().get_config()
        config["mass"] = self.mass
        
        return config

    def set_seed(self, seed):
        pass

    class Factory:
        def create(self): return VelocityModel()

In [29]:
#export
class DummyModel(BaseFunction):    
    def __init__(self, name="World", value=0, links=None, new_name=True, namespace=None, **cargs):        
        super().__init__(name=name, value=value, links=links, new_name=new_name, namespace=namespace)
    
    def __call__(self, verbose=False):        
        pass

    def summary(self):
        super().summary("")
    
    def get_config(self):
        pass

    class Factory:
        def create(self): return DummyModel()

### OpenAI Gym
An example showing how to use an OpenAI Gym function. And how to have another function which accesses one of the values of the gym environment. 

In [30]:
from pct.functions import Constant
from pct.functions import IndexedParameter
from pct.putils import FunctionsList
from pct.functions import Proportional

render=False 
print(render)
acrobot = OpenAIGym("Acrobot-v1", render=render)
namespace=acrobot.namespace
acrobot.add_link(Constant(1, namespace=namespace))
acrobot.get_config()

False


{'type': 'OpenAIGym',
 'name': 'gym',
 'value': 0,
 'links': {0: 'constant'},
 'env_name': 'Acrobot-v1',
 'reward': 0,
 'done': False,
 'info': {}}

In [31]:
acrobot()
print(acrobot.reward)
print(getattr(acrobot, "reward"))

-1.0
-1.0


In [32]:
acrobot.output_string()

'1.000 0.014 0.997 0.075 0.001 -0.114 -1.0 False {}'

In [33]:
acrobot.value[0]

0.99989915

The IndexedParameter type retrieves a value from a linked function based upon an index.

In [34]:
cos_angle1 = IndexedParameter(0, name="cos_angle1", namespace=namespace)
cos_angle1.add_link(acrobot)
print(cos_angle1.get_config())
cos_angle1()

{'type': 'IndexedParameter', 'name': 'cos_angle1', 'value': 0, 'links': {0: 'gym'}, 'index': 0}


0.99989915

In [35]:
acrobot.close()

In [36]:
#hide
FunctionsList.getInstance().get_function(name="cos_angle1", namespace=namespace).set_name("cos_angle2")
assert cos_angle1.get_name() == "cos_angle2"

In [37]:
#hide
pp = Proportional(10, namespace=namespace)
print(pp.get_config())
pp.set_property("gain", 3)
print(pp.get_config())

{'type': 'Proportional', 'name': 'proportional', 'value': 0, 'links': {}, 'gain': 10}
{'type': 'Proportional', 'name': 'proportional', 'value': 0, 'links': {}, 'gain': 3}


In [38]:
#gui
pen = Pendulum(render=True, namespace=namespace)
pen.add_link(Constant([1], namespace=namespace))
print(pen.get_config())
pen.run(steps=10, verbose=True)

{'type': 'Pendulum', 'name': 'Pendulum', 'value': 0, 'links': {0: 'constant1'}, 'env_name': 'Pendulum-v1', 'reward': 0, 'done': False, 'info': {}}
1.000 -0.030 -0.454 -0.030 9.970 [0.03691516] False {}
0.999 -0.046 -0.326 -0.046 9.954 [0.02250519] False {}
0.998 -0.057 -0.211 -0.057 9.943 [0.0137712] False {}
0.998 -0.062 -0.103 -0.062 9.938 [0.00864718] False {}
0.998 -0.062 0.000 -0.062 9.938 [0.00587735] False {}
0.998 -0.056 0.104 -0.056 9.944 [0.00480763] False {}
0.999 -0.046 0.212 -0.046 9.954 [0.00527731] False {}
1.000 -0.030 0.327 -0.030 9.970 [0.00759467] False {}
1.000 -0.007 0.455 -0.007 9.993 [0.01259255] False {}
1.000 0.023 0.600 0.023 10.023 [0.02177306] False {}


array([ 0.99972999,  0.02323734,  0.60019398,  0.02323881, 10.02323881])

In [39]:
#gui
pen.close()

/home/ruperty/anaconda3/envs/dev37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Velocity Model

In [40]:
vm = VelocityModel(name='VelocityModel', namespace=namespace)
vm.add_link(Constant(1, name='mycon'))
print(vm.summary())
config = vm.get_config()
print(config)
assert config == {'type': 'VelocityModel', 'name': 'VelocityModel', 'value': 0, 'links': {0: 'mycon'}, 'mass': 50}
vm.run(steps=10, verbose=True)

VelocityModel VelocityModel | 0 | links  mycon 
None
{'type': 'VelocityModel', 'name': 'VelocityModel', 'value': 0, 'links': {0: 'mycon'}, 'mass': 50}
0.020 0.040 0.060 0.080 0.100 0.120 0.140 0.160 0.180 0.200 

0.19999999999999998

In [41]:
#hide
#FunctionsList.getInstance().report()

In [42]:
#%nbdev_hide
#notebook2script()